In [20]:
#!pip install folium

In [21]:
#importing of packages
import pandas as pd
import folium
from folium.plugins import HeatMap
import re


In [22]:
# 1. Clean Postal Codes
def clean_postal_codes(df, column_name):
    """Clean and validate postal codes."""
    def clean_postal(postal):
        if pd.isna(postal):
            return None
        # Remove leading/trailing spaces, convert to uppercase, and remove invalid chars
        postal = re.sub(r"[^A-Z0-9]", "", postal.upper().strip())
        # Add space in the correct position if format matches
        if re.match(r"^[A-Z]\d[A-Z]\d[A-Z]\d$", postal):
            return postal[:3] + " " + postal[3:]
        return None  # Invalid postal code

    df[column_name] = df[column_name].apply(clean_postal)
    return df

# 2. Map Postal Codes to Latitude and Longitude
def map_postal_codes(df, postal_code_col, postal_mapping_path):
    """Map postal codes to their latitude and longitude using a reference file."""
    # Load the postal code mapping file
    postal_mapping = pd.read_csv(postal_mapping_path, on_bad_lines='skip')

    
    # Merge with survey data
    df = df.merge(postal_mapping, how="left", left_on=postal_code_col, right_on="POSTAL_CODE")
    
    # Group by postal codes and coordinates, then count
    grouped = df.groupby([postal_code_col, "LATITUDE", "LONGITUDE"]).size().reset_index(name="count")
    return grouped

# Main Script
if __name__ == "__main__":
    # File paths
    fake_data_output = "fake_postal_codes.csv"
    postal_mapping_file = "CanadianPostalCodes202403.csv"  # Latitude/Longitude data
    cleaned_output_file = "cleaned_postal_data.csv"


    # Load Fake Data and Clean
    fake_data = pd.read_csv(fake_data_output)
    fake_data_cleaned = clean_postal_codes(fake_data, 'postal_code')

    # Map to Latitude/Longitude
    postal_summary = map_postal_codes(fake_data_cleaned, 'postal_code', postal_mapping_file)


In [23]:
# Step 3: Create an interactive Folium map
map_center = [43.7315, -79.7604]  # Centered on Brampton, Ontario
brampton_map = folium.Map(location=map_center, zoom_start=11)

In [24]:
# Step 4: Add Circle Markers to the map
for _, row in postal_summary.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=row['count'] * 2,  # Marker size based on count
        color='blue',
        fill=True,
        fill_opacity=0.5,
        popup=f"<b>Postal Code:</b> {row['postal_code']}<br><b>Count:</b> {row['count']}"
    ).add_to(brampton_map)
